Import necessary modules

In [ ]:
from gpplus.models import GP_Plus
from gpplus.test_functions.multi_fidelity import multi_fidelity_wing
from gpplus.preprocessing import train_test_split_normalizeX
from gpplus.utils import set_seed

Data generation and prepration:The next step is to generate the data, standardize it and separate it to train-test sets.Here, we have 4 different sources and 'num' illustrates the number of samples from each of the sources.

To make the problem more challenging, we also add Gaussian noise to the sources with the corresponding standard deviation defined in 'noise_std'.

Then, we generate the data, standardize them and use 1% of that as the train and the rest for test.

In [ ]:
# Set parameters and generate data
set_seed(4)
qual_index = {10: 4}
num = {'0': 5000, '1': 10000, '2': 10000, '3': 10000}
noise_std = {'0': 0.5, '1': 1.0, '2': 1.5, '3': 2.0}
X, y = multi_fidelity_wing(n=num, noise_std=noise_std, random_state=4)

# Split and normalize data
Xtrain, Xtest, ytrain, ytest = train_test_split_normalizeX(X, y, test_size=0.99, qual_index_val=qual_index, stratify=X[..., list(qual_index.keys())])

Create and fit the GP+ model: Firstly, we create the model using GP_Plus command, and then the model is optimized with model.fit. 

Due to the multi-fidelity nature of this example, we set 'multiple_noise=True' to estimate separate noise process for each data source. Additionally, to improve the accuracy of the trained model we use multiple constant base as the mean of the model ('base='multiple_constant')

In [ ]:
model = GP_Plus(Xtrain, ytrain, qual_ind_lev=qual_index, multiple_noise=True, base='multiple_constant')
model.fit(n_jobs=-1)

Visualization and evaluation: As this is a multi-fidelity problem, we use the 'model.visualize_latent(type='MF')' to visualize the global correlation among fidelity sources (their learned manifold).  

Then, the accuarcy of the model is evaluated using 'model.evaluation()' command.

In [ ]:
model.evaluation(Xtest, ytest)
model.visualize_latent(type='MF')